In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

from sklearn.pipeline import Pipeline
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV,cross_validate,KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
from scipy.stats import uniform

In [ ]:
df=pd.read_excel("/kaggle/input/mmm-dataset/MMM Dataset.xlsx")

In [ ]:
df.head(10)

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
numerical_columns = [ 'DEMAND ',
       'Consumer Price Index (CPI)', 'Consumer Confidence Index(CCI)',
       'Producer Price Index (PPI)', 'Unit Price ($)', 'POS/ Supply Data',
       'SALES ($)', 'Advertising Expenses (SMS)',
       'Advertising Expenses(Newspaper ads)', 'Advertising Expenses(Radio)',
       'Advertising Expenses(TV)', 'Advertising Expenses(Internet)',
       'GRP (NewPaper ads)', 'GRP(SMS)', 'GRP(Radio', 'GRP(Internet)',
       'GRP(TV)']

In [ ]:
media= ['Advertising Expenses (SMS)', 'Advertising Expenses(Newspaper ads)', 
                  'Advertising Expenses(Radio)', 'Advertising Expenses(TV)', 
                  'Advertising Expenses(Internet)']

In [ ]:
df_drop=df.drop(['TV Manufacturing Brand', 'DATE'],axis=1)

In [ ]:
corr_matrix = df_drop.corr()
plt.figure(figsize=(5,5))
sns.clustermap(corr_matrix, annot = True, fmt = ".2f")
plt.title("Correlation between columns")
plt.show

In [ ]:
for i, col in enumerate(numerical_columns, 1):
    plt.subplot(len(numerical_columns), 1, i)
    sns.boxplot(x=col, data=df)

plt.show()

In [ ]:
#capping outliers
def cap_outliers(df, column):
    lower_bound = np.percentile(df[column], 5)
    upper_bound = np.percentile(df[column], 95)
    df[column] = np.where(df[column] < lower_bound, lower_bound, df[column])
    df[column] = np.where(df[column] > upper_bound, upper_bound, df[column])
    return df

In [ ]:
outliers = ['DEMAND ', 'Unit Price ($)', 'POS/ Supply Data', 'SALES ($)',
            'Advertising Expenses(Radio)',
       'Advertising Expenses(TV)', 'Advertising Expenses(Internet)',
            'GRP(SMS)', 'GRP(Radio', 'GRP(Internet)',
       'GRP(TV)']

In [ ]:
for col in outliers:
    cap_outliers(df, col)

In [ ]:
for i, col in enumerate(numerical_columns, 1):
    plt.subplot(len(numerical_columns), 1, i)
    sns.boxplot(x=col, data=df)

plt.show()

In [ ]:
from scipy import stats

# Function to remove outliers based on Z-score
def remove_outliers_zscore(df, column):
    z_scores = np.abs(stats.zscore(df[column]))
    df = df[z_scores < 3]  # Remove rows where Z-score is greater than 3
    return df

# Apply Z-score outlier removal to relevant columns
for col in outliers:
    df = remove_outliers_zscore(df, col)


In [ ]:
for i, col in enumerate(numerical_columns, 1):
    plt.subplot(len(numerical_columns), 1, i)
    sns.boxplot(x=col, data=df)

plt.show()

In [ ]:
#duplicate values
duplicates = df.duplicated()
print("Number of duplicate rows:", duplicates.sum())

In [ ]:
df.columns

In [ ]:
df['DATE'] = sorted(df['DATE'])

In [ ]:
media = ['Advertising Expenses (SMS)',
       'Advertising Expenses(Newspaper ads)', 'Advertising Expenses(Radio)',
       'Advertising Expenses(TV)', 'Advertising Expenses(Internet)']
media

In [ ]:
lag_periods = [1, 2]
for col in media:
        for lag in lag_periods:
            df[f'{col}_Lag{lag}'] = df[col].shift(lag)

In [ ]:
df.head(10)

In [ ]:
#interaction terms
for i, channel1 in enumerate(media):
    for channel2 in media[i+1:]:
        df[f'{channel1}_{channel2}_interaction'] = df[channel1] * df[channel2]

In [ ]:
# Normalization
scaler = StandardScaler()
df[media] = scaler.fit_transform(df[media])

In [ ]:
df['Day'] = df['DATE'].dt.day
df['Month'] = df['DATE'].dt.month
df['Year'] = df['DATE'].dt.year

In [ ]:
df.columns

In [ ]:
# Define the columns to drop
columns_to_drop = [
    'Advertising Expenses (SMS)_Advertising Expenses(Newspaper ads)_interaction',
    'Advertising Expenses (SMS)_Advertising Expenses(Radio)_interaction',
    'Advertising Expenses (SMS)_Advertising Expenses(TV)_interaction',
    'Advertising Expenses (SMS)_Advertising Expenses(Internet)_interaction',
    'Advertising Expenses(Newspaper ads)_Advertising Expenses(Radio)_interaction',
    'Advertising Expenses(Newspaper ads)_Advertising Expenses(TV)_interaction',
    'Advertising Expenses(Newspaper ads)_Advertising Expenses(Internet)_interaction',
    'Advertising Expenses(Radio)_Advertising Expenses(TV)_interaction',
    'Advertising Expenses(Radio)_Advertising Expenses(Internet)_interaction',
    'Advertising Expenses(TV)_Advertising Expenses(Internet)_interaction',
    'Advertising Expenses (SMS)_Lag1',
    'Advertising Expenses (SMS)_Lag2',
    'Advertising Expenses(Newspaper ads)_Lag1',
    'Advertising Expenses(Newspaper ads)_Lag2',
    'Advertising Expenses(Radio)_Lag1',
    'Advertising Expenses(Radio)_Lag2',
    'Advertising Expenses(TV)_Lag1',
    'Advertising Expenses(TV)_Lag2',
    'Advertising Expenses(Internet)_Lag1',
    'Advertising Expenses(Internet)_Lag2',
    'TV Manufacturing Brand',
    'DATE',
    'SALES ($)']
​
# Drop the columns
df_drop = df.drop([col for col in columns_to_drop if col in df.columns], axis=1)
df_drop.columns

In [ ]:
to_drop = ['TV Manufacturing Brand',
    'DATE',
    'SALES ($)']
# Drop the columns
df_new = df.drop([col for col in to_drop if col in df.columns], axis=1)
df_new.columns

In [ ]:
df_new.update(df_new[['Advertising Expenses (SMS)_Lag1',
       'Advertising Expenses (SMS)_Lag2',
       'Advertising Expenses(Newspaper ads)_Lag1',
       'Advertising Expenses(Newspaper ads)_Lag2',
       'Advertising Expenses(Radio)_Lag1', 'Advertising Expenses(Radio)_Lag2',
       'Advertising Expenses(TV)_Lag1', 'Advertising Expenses(TV)_Lag2',
       'Advertising Expenses(Internet)_Lag1',
       'Advertising Expenses(Internet)_Lag2']].fillna(0))


In [ ]:
df_new.isnull().sum()

In [ ]:
#X = df[media + [col for col in df.columns if 'lag' in col or 'interaction' in col] + 
#      ['Consumer Price Index (CPI)', 'Consumer Confidence Index(CCI)', 'Producer Price Index (PPI)',
#        'Day', 'Month', 'Year']]
X = df_new
y = df['SALES ($)']

In [ ]:
df_drop.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0,max_iter=10000),
    'Lasso Regression': Lasso(alpha=1.0,max_iter=10000)
}

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [ ]:
results={}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    results[name] = {'R2': r2, 'RMSE': rmse, 'mae':mae}
results

In [ ]:
n_folds = 5

In [ ]:
results_kfold = {name: {'R2': [], 'RMSE': [], 'MAE': []} for name in models.keys()}

# k-Fold Cross-Validation for overfitting
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
for name, model in models.items():
    for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
        X_fold_train, X_fold_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_fold_train, y_fold_val = y_train.iloc[train_index], y_train.iloc[val_index]
        
        model.fit(X_fold_train, y_fold_train)
        y_pred = model.predict(X_fold_val)
        
        r2 = r2_score(y_fold_val, y_pred)
        rmse = np.sqrt(mean_squared_error(y_fold_val, y_pred))
        mae = mean_absolute_error(y_fold_val, y_pred)
        
        results_kfold[name]['R2'].append(r2)
        results_kfold[name]['RMSE'].append(rmse)
        results_kfold[name]['MAE'].append(mae)
    
    #mean across folds for each metric
    results_kfold[name]['R2'] = np.mean(results_kfold[name]['R2'])
    results_kfold[name]['RMSE'] = np.mean(results_kfold[name]['RMSE'])
    results_kfold[name]['MAE'] = np.mean(results_kfold[name]['MAE'])

print("\nk-Fold Cross-Validation Results (Mean across {} folds):".format(n_folds))
results_kfold

In [ ]:
coefficients={}
for name, model in models.items():
    if hasattr(model, 'coef_'):
        coefficients[name] = model.coef_

In [ ]:
media_channel_indices = [X_train.columns.get_loc(channel) for channel in media]
for name, coef in coefficients.items():
    media_coefficients = coef[media_channel_indices] 
    print(f"Coefficients for {name} (Media Channels):")
    for channel, coef_value in zip(media, media_coefficients):
        print(f"  {channel}: {coef_value}")

In [ ]:
ridge_coeffs = {
    'Advertising Expenses (SMS)': 9850.88438573013,
  'Advertising Expenses(Newspaper ads)': 4550.927789763226,
  'Advertising Expenses(Radio)': -11283.211745332117,
  'Advertising Expenses(TV)': 9027.253290668077,
  'Advertising Expenses(Internet)': -3378.488745396707,
}

In [ ]:
sales = df['SALES ($)']
def calculate_elasticity(coefficients, media,sales):
    #mean of media spend for each channel
    mean_media_spend = df[media].mean()
    
    #mean of sales
    mean_sales = y.mean()
    
    # Elasticity dictionary
    elasticity = {}
    
    for channel, coeff in coefficients.items():
        elasticity[channel] = (coeff * mean_media_spend[channel]) / mean_sales
    
    return elasticity

elasticities = calculate_elasticity(ridge_coeffs, media, sales)

# Print results
print("Elasticities for each media channel:")
for x, elasticity_value in elasticities.items():
    print(f"{x}: {elasticity_value*1000000000000000000:.17f}")

In [ ]:
total_budget = 500000
#negative coefficients to zero
for channel, coeff in ridge_coeffs.items():
    if coeff < 0:
        ridge_coeffs[channel] = 0

total_coeff_sum = sum(ridge_coeffs.values())
# budget allocation based on normalized coefficients
allocated_budget = {channel: (coeff / total_coeff_sum) * total_budget for channel,
                    coeff in ridge_coeffs.items()}
#10% min allocation
min_allocation = total_budget * 0.1

def adjust_budget_allocation(budgets, total_budget, min_allocation):
    adjusted_budgets = budgets.copy()
    for channel, budget in adjusted_budgets.items():
        if budget < min_allocation:
            adjusted_budgets[channel] = min_allocation

    # Redistribute the remaining budget
    remaining_budget = total_budget - sum(adjusted_budgets.values())
    if remaining_budget > 0:
        remaining_coeff_sum = sum(budget for budget in budgets.values() if budget >= min_allocation)
        for channel, budget in adjusted_budgets.items():
            if budget >= min_allocation:
                adjusted_budgets[channel] += (budgets[channel] / remaining_coeff_sum) * remaining_budget

    return adjusted_budgets

final_budget_allocation = adjust_budget_allocation(allocated_budget, total_budget, min_allocation)

print("\nFinal Budget Allocation with Minimum Constraints:")
for channel, budget in final_budget_allocation.items():
    print(f"{channel}: Rs {budget:.2f}")


In [ ]:
# Sales over time
plt.figure(figsize=(12, 6))
plt.plot(df['DATE'], df['SALES ($)'], label='Sales')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.title('Sales Over Time')
plt.legend()
plt.show()

In [ ]:
# Media spend across different channels
plt.figure(figsize=(12, 6))
for channel in media:
    plt.bar(df['DATE'], df[channel], label=channel)
plt.xlabel('Date')
plt.ylabel('Media Spend ($)')
plt.title('Media Spend Across Different Channels')
plt.legend()
plt.show()

In [ ]:
# Plot optimal spend allocation
plt.figure(figsize=(18, 10))
channels = list(final_budget_allocation.keys())
allocations = list(final_budget_allocation.values())
plt.bar(channels, allocations, color='brown')
plt.xlabel('Media Channels')
plt.ylabel('Optimal Spend (Rs)')
plt.title('Optimal Media Spend Allocation')
plt.show()

In [ ]:
# Extract media channel coefficients
media_coefficients = coefficients[coefficients['Feature'].isin(media)]

# Plot contribution of each media channel to sales
plt.figure(figsize=(12, 6))
media_coefficients.plot(kind='bar', x='Feature', y='Coefficient', legend=False, color='purple')
plt.xlabel('Media Channels')
plt.ylabel('Coefficient')
plt.title('Contribution of Each Media Channel to Sales')
plt.show()